## Introduction

This is an example to derive SMIRNOFF typed atom-centerd polarizabilities from quantum mechanically calculated electrostatics potentials with `factorpol` package.

A global optimization is built by concatenating Individual objective function find polarizabilities for the training set.

## Dependencies

- ray
- openff-toolkit
- openff-recharge*
- sqlalchemy
- openeye-toolkits
- scipy

In [1]:
import os
import pickle
from glob import glob

import numpy as np
import pandas as pd
from openff.toolkit import ForceField, Molecule
from pkg_resources import resource_filename

from factorpol.alphas_training import (AlphaData, AlphasTrainer, AlphaWorker,
                                       optimize_alphas, optimize_alphas_fast)
from factorpol.charge_training import ChargeTrainer
from factorpol.utilities import (calc_rrms, flatten_a_list, Polarizability,
                                 StorageHandler)

cwd = os.getcwd()

/home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


## Prepare dataset

Curate QM ESPs data generated in `00-generate-qm-reference.ipynb`.

`off_examples.offxml` is an example OpenForceField style [ForceField file](https://github.com/openforcefield/openff-forcefields/tree/main/openforcefields/offxml). We use the `<vdW>` handeler to label atoms with SMIRNOFF patterns and assign electrostatics parameters until we have a new handeler for polarizability.


In [2]:
off_forcefield = ForceField(resource_filename(
    "factorpol", os.path.join("data", "off_examples.offxml")
))

# Initialize a polarizability 
alphas0 = Polarizability()

Curate QM data and prepare ray workers to optimize polarizabilities

In [3]:
data = AlphaData(
    database_name="factorpol_examples",
    dataset=["CO", "C=C"],
    off_forcefield=off_forcefield,
    polarizability=alphas0,
    num_cpus=2,
)

2023-04-14 14:08:52,447	INFO worker.py:1553 -- Started a local Ray instance.
(pid=1848527) /home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=1848527)   setattr(self, word, getattr(machar, word).flat[0])
(pid=1848527) /home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=1848527)   return self._float_to_str(self.smallest_subnormal)
(pid=1848527) /home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
(pid=1848527)   setattr(self, word, getattr(machar, word).flat[0])
(pid=1848527) /home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py

Because we have 2 molecules in the training set, each of them has two sets of QM ESPs, which means 1 workers per molecule and a totoal of 2 workers.

In [4]:
print(f"Number of data in training set:\t {len(data.workers)}")

Number of data in training set:	 2


## Optimization

In [5]:
atrain = AlphasTrainer(
    workers=data.workers,
    prior=alphas0,
    working_directory=os.path.join(cwd, "data_alphas_2"),
)

Path exists, deleting


In [6]:
ret = optimize_alphas(worker_list=atrain.workers, solved=True, num_cpus=8)

2023-04-14 14:08:58,406	INFO worker.py:1553 -- Started a local Ray instance.
(pid=1850715) /home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=1850715)   setattr(self, word, getattr(machar, word).flat[0])
(pid=1850715) /home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=1850715)   return self._float_to_str(self.smallest_subnormal)
(pid=1850715) /home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
(pid=1850715)   setattr(self, word, getattr(machar, word).flat[0])
(pid=1850715) /home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py

## Results

In [7]:
ret

{'[#6:1]': 1.7531861665519723 <Unit('angstrom ** 3')>,
 '[#8:1]': 0.5093480935921064 <Unit('angstrom ** 3')>,
 '[#1:1]': 0.17091150224579515 <Unit('angstrom ** 3')>}

## A faster method to optimize polarizabilities
**This method is extremely experimental and not recommended for production use**

In [8]:
ret2 = optimize_alphas_fast(worker_list=atrain.workers, solved=True, num_cpus=8)

2023-04-14 14:09:03,630	INFO worker.py:1553 -- Started a local Ray instance.
(pid=1853171) /home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=1853171)   setattr(self, word, getattr(machar, word).flat[0])
(pid=1853171) /home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=1853171)   return self._float_to_str(self.smallest_subnormal)
(pid=1853171) /home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
(pid=1853171)   setattr(self, word, getattr(machar, word).flat[0])
(pid=1853171) /home/wwilla/mambaforge/envs/factorpol-dev/lib/python3.9/site-packages/numpy/core/getlimits.py

## Results

In [9]:
ret2

{'[#6:1]': 1.7531861665519999 <Unit('angstrom ** 3')>,
 '[#8:1]': 0.5093480935921196 <Unit('angstrom ** 3')>,
 '[#1:1]': 0.1709115022457837 <Unit('angstrom ** 3')>}